<a href="https://colab.research.google.com/github/TokyoYoshida/GoogleColab/blob/master/playground_deep_learning_keiba.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
# 必要ならば以下のようにディレクトリ移動する
 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd /content/drive/'My Drive'/'★個人専用ファイル'/'開発用・個人サービス開発'/'GoogleColab'/

/content/drive/My Drive/★個人専用ファイル/開発用・個人サービス開発/GoogleColab


In [3]:
%ls


data/
duel_dqn_Pendulum-v0_weights.h5f
h5f/
model.txt
playground_deep_learning_keiba
playground_dqn_keiba
playground_dqn_keiba_try_layer_add_depth
playground_dqn_keiba_try_long_term_reward
playground_dqn_keiba_try_simple
playground_dqn_keiba_try_simple2
playground_dqn_keiba_try_simple_x_1
playground_dqn_study.ipynb
playground_dqn_study_my_try.ipynb
playground_dqn_study_my_try_modify_reward
playground.ipynb


In [0]:
import pandas as pd

In [0]:
import numpy as np

In [0]:
from pprint import pprint

In [7]:
!pip install tensorflow==2.2.0

In [0]:
!pip install -q keras

In [9]:
import keras
from keras.utils import np_utils


Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import RMSprop


In [0]:
# 競馬データの作成

In [0]:
ldat = pd.read_pickle("./data/keiba_dat_res_20190611_sorted.pkl")

In [54]:
ldat.head(1)

,rank,frame_num,horse_num,name,sex,age,amount,jockey,time,diff_arrival,odds,popularity,weight,diff_weight,trainer,race_num,grade,ground,direction,unknown,distance,weather,condition,start_time,year,month,day,prev_rank,prev_frame_num,prev_horse_num,prev_name,prev_sex,prev_age,prev_amount,prev_jockey,prev_time,prev_diff_arrival,prev_odds,prev_popularity,prev_weight,prev_diff_weight,prev_trainer,prev_race_num,prev_grade,prev_ground,prev_direction,prev_unknown,prev_distance,prev_weather,prev_condition,prev_start_time,prev_year,prev_month,prev_day,winner,return,prev_time_float,start_time_float,prev_start_time_float
0,1.0,6,6,0,0,2,54.0,0,1:31.6,0,3.2,1.0,474.0,0.0,0,1,9,0,0,-,1500.0,0,0,10:40,2006,8,12,3.0,6.0,6.0,0.0,0.0,2.0,51.0,75.0,1:12.2,1.0,9.6,3.0,474.0,-2.0,0.0,1.0,9.0,0.0,0.0,-,1200.0,1.0,0.0,10:40,2006.0,7.0,29.0,True,3.2,1561165932,1561200000,1561200000


,rank,frame_num,horse_num,name,sex,age,amount,jockey,time,diff_arrival,odds,popularity,weight,diff_weight,trainer,race_num,grade,ground,direction,unknown,distance,weather,condition,start_time,year,month,day,prev_rank,prev_frame_num,prev_horse_num,prev_name,prev_sex,prev_age,prev_amount,prev_jockey,prev_time,prev_diff_arrival,prev_odds,prev_popularity,prev_weight,prev_diff_weight,prev_trainer,prev_race_num,prev_grade,prev_ground,prev_direction,prev_unknown,prev_distance,prev_weather,prev_condition,prev_start_time,prev_year,prev_month,prev_day,winner,return,prev_time_float,start_time_float,prev_start_time_float
0,11.0,7,13,14140,1,5,57.0,93,1:25.0,1,60.4,12.0,466.0,0.0,142,9,9,1,0,-,1400.0,0,1,14:50,2010,4,18,1.0,4.0,7.0,14140.0,1.0,5.0,57.0,253.0,1:12.2,0.0,3.9,2.0,466.0,0.0,142.0,12.0,9.0,1.0,0.0,-,1200.0,0.0,0.0,16:40,2010.0,3.0,28.0,False,0.0,1561165932,1561215000,1561221600
1,16.0,3,6,26677,0,3,53.0,336,1:17.6,16,41.0,9.0,482.0,0.0,392,1,9,1,0,-,1200.0,0,2,10:20,2010,7,4,16.0,4.0,8.0,26677.0,0.0,3.0,54.0,7.0,1:18.1,13.0,50.4,8.0,482.0,0.0,392.0,1.0,9.0,1.0,0.0,-,1200.0,0.0,0.0,10:15,2010.0,6.0,6.0,False,0.0,1561166286,1561198800,1561198500
2,6.0,3,3,28580,0,5,55.0,80,2:01.3,6,54.1,10.0,458.0,6.0,35,8,9,0,2,外,2000.0,1,0,13:30,2013,10,27,17.0,6.0,12.0,28580.0,0.0,5.0,55.0,68.0,1:49.0,6.0,62.8,14.0,452.0,0.0,35.0,12.0,9.0,0.0,2.0,外,1800.0,1.0,0.0,16:30,2013.0,7.0,27.0,False,0.0,1561168140,1561210200,1561221000
3,2.0,1,1,40872,1,4,57.0,194,1:52.2,12,12.7,7.0,464.0,12.0,123,7,9,1,0,-,1800.0,1,0,13:15,2015,10,11,5.0,4.0,6.0,40872.0,1.0,4.0,57.0,194.0,1:53.7,9.0,20.9,6.0,452.0,2.0,123.0,12.0,9.0,1.0,0.0,-,1800.0,0.0,0.0,16:10,2015.0,9.0,13.0,False,0.0,1561168422,1561209300,1561219800
4,10.0,3,5,31495,1,2,55.0,408,1:57.9,17,440.3,13.0,440.0,2.0,132,3,9,1,0,-,1800.0,0,0,11:00,2011,11,12,11.0,8.0,15.0,31495.0,1.0,2.0,54.0,336.0,1:29.1,12.0,333.9,14.0,438.0,2.0,132.0,2.0,9.0,1.0,0.0,-,1400.0,0.0,0.0,10:30,2011.0,10.0,9.0,False,0.0,1561166946,1561201200,1561199400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
542850,10.0,2,4,50194,0,3,54.0,249,1:15.0,12,402.3,14.0,410.0,4.0,133,1,9,1,0,-,1200.0,0,0,10:10,2016,1,17,9.0,6.0,11.0,50194.0,0.0,2.0,54.0,249.0,1:15.3,1.0,349.8,14.0,406.0,-4.0,133.0,1.0,9.0,1.0,0.0,-,1200.0,0.0,0.0,10:05,2015.0,12.0,20.0,False,0.0,1561166118,1561198200,1561197900
542851,9.0,2,3,11165,1,2,53.0,96,1:28.8,1,16.4,7.0,446.0,-6.0,57,1,9,1,2,-,1400.0,0,0,10:00,2007,10,6,11.0,7.0,10.0,11165.0,1.0,2.0,54.0,89.0,1:13.1,4.0,15.5,7.0,452.0,4.0,57.0,1.0,9.0,0.0,0.0,外,1200.0,3.0,0.0,10:05,2007.0,9.0,30.0,False,0.0,1561165986,1561197600,1561197900
542852,3.0,7,13,29079,0,5,55.0,83,1:09.0,3,16.9,7.0,466.0,2.0,15,12,9,0,0,-,1200.0,0,0,16:30,2014,7,12,6.0,6.0,12.0,29079.0,0.0,5.0,55.0,246.0,1:22.8,4.0,6.3,5.0,464.0,6.0,15.0,12.0,9.0,0.0,2.0,-,1400.0,0.0,0.0,16:01,2014.0,5.0,18.0,False,0.0,1561166568,1561221000,1561219260
542853,9.0,6,12,30443,1,4,57.0,18,1:14.6,3,280.2,16.0,446.0,-12.0,197,9,9,1,2,-,1200.0,3,0,14:15,2013,3,10,10.0,5.0,7.0,30443.0,1.0,4.0,57.0,18.0,0:59.4,4.0,126.7,12.0,458.0,6.0,197.0,5.0,9.0,1.0,0.0,-,1000.0,1.0,2.0,11:50,2013.0,2.0,16.0,False,0.0,1561165164,1561212900,1561204200


In [0]:
sdat = ldat.sample(frac=1).reset_index(drop=True)

In [0]:
# 勝ち馬と負け馬をちょうど半々になるようにする start

In [0]:
winner = sdat.query("rank == 1")

In [0]:
loser = sdat.query("rank != 1").head(winner.count()[0])

In [0]:
catted = pd.concat([winner,loser])

In [0]:
sampled = catted.sample(frac=1).reset_index(drop=True)

In [121]:
# 勝ち馬と負け馬をちょうど半々になるようにする end

,rank,frame_num,horse_num,name,sex,age,amount,jockey,time,diff_arrival,odds,popularity,weight,diff_weight,trainer,race_num,grade,ground,direction,unknown,distance,weather,condition,start_time,year,month,day,prev_rank,prev_frame_num,prev_horse_num,prev_name,prev_sex,prev_age,prev_amount,prev_jockey,prev_time,prev_diff_arrival,prev_odds,prev_popularity,prev_weight,prev_diff_weight,prev_trainer,prev_race_num,prev_grade,prev_ground,prev_direction,prev_unknown,prev_distance,prev_weather,prev_condition,prev_start_time,prev_year,prev_month,prev_day,winner,return,prev_time_float,start_time_float,prev_start_time_float
0,14.0,4,8,52294,0,3,54.0,52,1:58.5,11,209.3,15.0,412.0,-4.0,81,3,9,1,0,-,1800.0,0,2,11:10,2016,9,17,14.0,8.0,16.0,52294.0,0.0,3.0,54.0,52.0,0:56.2,12.0,104.8,17.0,416.0,-4.0,81.0,7.0,9.0,0.0,3.0,-,NaN,0.0,0.0,13:30,2016.0,8.0,13.0,False,0.0,1561164972,1561201800,1561210200
1,14.0,6,12,7038,1,3,56.0,6,1:48.7,2,123.8,13.0,482.0,-2.0,254,9,9,1,2,-,1700.0,0,0,14:25,2007,12,16,6.0,3.0,3.0,7038.0,1.0,3.0,55.0,44.0,1:55.6,2.0,110.8,8.0,484.0,0.0,254.0,7.0,9.0,1.0,0.0,-,1800.0,0.0,0.0,13:20,2007.0,11.0,24.0,False,0.0,1561168536,1561213500,1561209600
2,1.0,3,4,6549,1,5,57.0,168,1:09.0,0,12.4,5.0,508.0,-6.0,249,6,9,0,0,-,1200.0,0,0,12:55,2007,4,14,11.0,3.0,5.0,6549.0,1.0,5.0,57.0,194.0,1:10.1,4.0,8.4,4.0,514.0,22.0,249.0,12.0,9.0,0.0,0.0,-,1200.0,0.0,0.0,16:05,2007.0,2.0,10.0,True,12.4,1561165806,1561208100,1561219500
3,1.0,5,7,17422,1,3,55.0,253,0:59.1,0,1.2,1.0,466.0,-2.0,52,2,9,1,0,-,1000.0,1,0,10:50,2008,9,7,2.0,3.0,3.0,17422.0,1.0,3.0,55.0,253.0,0:58.2,4.0,3.3,2.0,468.0,0.0,52.0,5.0,9.0,1.0,0.0,-,1000.0,1.0,2.0,12:40,2008.0,8.0,30.0,True,1.2,1561165092,1561200600,1561207200
4,4.0,6,11,41488,0,4,52.0,479,1:53.2,6,5.4,3.0,472.0,2.0,141,7,9,1,0,-,1800.0,0,0,13:25,2015,11,21,4.0,7.0,9.0,41488.0,0.0,4.0,54.0,444.0,1:55.0,3.0,3.1,1.0,470.0,-6.0,141.0,8.0,9.0,1.0,2.0,-,1800.0,0.0,1.0,13:35,2015.0,10.0,25.0,False,0.0,1561168500,1561209900,1561210500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75197,1.0,1,2,27628,0,3,54.0,14,1:24.4,0,4.2,2.0,444.0,2.0,10,12,9,1,2,-,1400.0,3,3,16:05,2011,11,19,1.0,2.0,2.0,27628.0,0.0,3.0,54.0,14.0,1:24.1,0.0,3.2,2.0,442.0,-2.0,10.0,7.0,9.0,1.0,0.0,-,1400.0,1.0,1.0,13:30,2011.0,11.0,6.0,True,4.2,1561166646,1561219500,1561210200
75198,1.0,7,7,53328,1,2,54.0,195,1:47.8,0,4.2,2.0,510.0,0.0,151,5,9,0,0,外,1800.0,1,0,12:15,2016,9,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,4.2,-9223372037,1561205700,-9223372037
75199,1.0,7,12,15180,0,4,55.0,200,1:47.4,0,9.9,5.0,434.0,-2.0,328,9,9,1,0,-,1700.0,1,1,14:10,2009,11,21,10.0,7.0,12.0,15180.0,0.0,4.0,55.0,44.0,1:48.0,3.0,55.0,11.0,436.0,8.0,328.0,3.0,9.0,1.0,0.0,-,1700.0,1.0,0.0,10:40,2009.0,10.0,25.0,True,9.9,1561168080,1561212600,1561200000
75200,1.0,8,12,13818,0,2,54.0,21,1:34.1,0,1.6,1.0,488.0,-10.0,141,2,9,0,0,-,1600.0,0,0,10:40,2008,10,18,3.0,2.0,2.0,13818.0,0.0,2.0,54.0,21.0,1:50.2,12.0,4.7,2.0,498.0,0.0,141.0,4.0,9.0,0.0,0.0,-,1800.0,1.0,0.0,12:30,2008.0,8.0,24.0,True,1.6,1561168212,1561200000,1561206600


In [0]:
x = sampled[['frame_num', 'horse_num', 'name', 'sex', 'age', 'amount',
       'jockey', 'odds', 'popularity', 'weight',
       'diff_weight', 'trainer', 'race_num', 'grade', 'ground', 'direction',
        'distance', 'weather', 'condition', 'start_time_float', 'year',
       'month', 'day', 'prev_rank', 'prev_frame_num', 'prev_horse_num',
       'prev_name', 'prev_sex', 'prev_age', 'prev_amount', 'prev_jockey',
       'prev_time_float', 'prev_diff_arrival', 'prev_odds', 'prev_popularity',
       'prev_weight', 'prev_diff_weight', 'prev_trainer', 'prev_race_num',
       'prev_grade', 'prev_ground', 'prev_direction',
       'prev_distance', 'prev_weather', 'prev_condition', 'prev_start_time_float',
       'prev_year', 'prev_month', 'prev_day']].dropna(0)

In [0]:
# x = sdat[['frame_num', 'horse_num', 'sex', 'age', 'amount',
#        'odds', 'popularity', 'weight',
#        'diff_weight', 'trainer', 'grade', 'ground', 'direction',
#         'distance', 'weather', 'condition', 'start_time_float', 'prev_rank', 'prev_frame_num', 'prev_horse_num',
#        'prev_sex', 'prev_age', 'prev_amount', 'prev_jockey',
#        'prev_time_float', 'prev_diff_arrival', 'prev_odds', 'prev_popularity',
#        'prev_weight', 'prev_diff_weight', 'prev_trainer',
#        'prev_grade', 'prev_ground', 'prev_direction',
#        'prev_distance', 'prev_weather', 'prev_condition', 'prev_start_time_float'
#        ]].dropna(0)

In [0]:
# x = sdat[["odds",'prev_rank','prev_diff_arrival']].dropna(0)

In [0]:
# x = sdat[["odds"]].dropna(0)

In [0]:
y = sampled[["return"]].dropna(0)

In [0]:
y_np = np_utils.to_categorical(sampled[["winner"]].dropna(0))
y = pd.DataFrame(y_np,columns=["lose","win"])

In [0]:
# 競馬データの作成 終わり

In [0]:
# 学習、テストデータの作成

In [0]:
trainlen = int(len(x)/2)

In [0]:
trainx = x.head(trainlen)
trainy = y.head(trainlen)

In [0]:
testx = x.tail(len(x)-trainlen)
testy = y.tail(len(x)-trainlen)

In [0]:
# 学習、テストデータの作成　終わり

In [0]:
#!pip uninstall -y keras

In [0]:
#!pip uninstall -y tensorflow

In [0]:
batch_size = 128
num_classes = 2
epochs = 200


In [0]:
model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(x.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2)) # 出力層2ノード,全結合
model.add(Activation("sigmoid"))

In [140]:
model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 512)               25600     
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 1026      
_________________________________________________________________
activation_3 (Activation)    (None, 2)                 0         
Total params: 289,282
Trainable params: 289,282
Non-trainable params: 0
________________________________________________

In [0]:
model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])


In [0]:
history = model.fit(trainx, trainy,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(testx, testy))


Train on 31960 samples, validate on 31960 samples
Epoch 1/200
31960/31960 [==============================] - 4s 119us/step - loss: 183.2098 - accuracy: 0.5051 - val_loss: 0.6932 - val_accuracy: 0.4965
Epoch 2/200
31960/31960 [==============================] - 4s 113us/step - loss: 58.5574 - accuracy: 0.5053 - val_loss: 0.6933 - val_accuracy: 0.4965
Epoch 3/200
31960/31960 [==============================] - 4s 113us/step - loss: 26.4396 - accuracy: 0.5052 - val_loss: 0.6933 - val_accuracy: 0.4965
Epoch 4/200
31960/31960 [==============================] - 4s 118us/step - loss: 44.9092 - accuracy: 0.5053 - val_loss: 0.6933 - val_accuracy: 0.4965
Epoch 5/200
31960/31960 [==============================] - 4s 118us/step - loss: 97.7932 - accuracy: 0.5054 - val_loss: 0.6932 - val_accuracy: 0.4965
Epoch 6/200
23552/31960 [=====================>........] - ETA: 0s - loss: 37.2702 - accuracy: 0.5036

In [0]:
predictions = model.predict(testx)

In [0]:
print(predictions)

In [0]:
score = model.evaluate(testx, testy, verbose=1)


In [0]:

print("Test score", score[0])
print("Test accuracy", score[1])

In [0]:
testy.head(100).tail(20)

In [0]:
pd.DataFrame(predictions,columns=["lose","win"]).query("win > 0.1")